# Processing Codenames Clues

In [1]:
from itertools import chain, combinations
import gensim

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True, limit=500000
)

In [3]:
def powerset(iterable):
    s = list(iterable)
    pwrset = list(chain.from_iterable(combinations(s, r) for r in range(len(s))))
    pwrset.remove(())
    return pwrset

In [4]:
blue = ['boot', 'chest', 'parachute', 'opera', 'square', 'triangle', 'watch', 'scientist', 'embassy']
red = ['duck', 'horn', 'club', 'knight', 'green', 'jet', 'ketchup', 'slug']
bomb = ['ship']

In [5]:
model.most_similar(
    positive=blue,
    negative=(red+bomb),
    restrict_vocab=50000)

[('sociologist', 0.2902722954750061),
 ('doctoral_student', 0.26502445340156555),
 ('Embassy', 0.2545923590660095),
 ('Associate_Professor', 0.24647679924964905),
 ('Observatory', 0.24054788053035736),
 ('researcher', 0.23794464766979218),
 ('collaborators', 0.23586289584636688),
 ('assistant_professor', 0.233751118183136),
 ('consulates', 0.23194798827171326),
 ('fractures', 0.230932354927063)]

In [14]:
combinations_red = powerset(red)[9:]

clue_options_red = [(subset_red, model.most_similar(
    positive=subset_red,
    restrict_vocab=50000)[0]) for subset_red in combinations_red]

list(sorted(clue_options_red, key=lambda r: r[1][1], reverse=True))

[(('jet', 'ketchup'), ('jets', 0.6368544101715088)),
 (('club', 'jet'), ('jets', 0.6173892021179199)),
 (('horn', 'jet'), ('jets', 0.6106443405151367)),
 (('duck', 'jet'), ('plane', 0.6101616621017456)),
 (('jet', 'slug'), ('jets', 0.5998554229736328)),
 (('green', 'jet'), ('jets', 0.5959645509719849)),
 (('club', 'slug'), ('clubs', 0.5954948663711548)),
 (('horn', 'club'), ('clubs', 0.594120442867279)),
 (('duck', 'ketchup'), ('chicken', 0.5847111940383911)),
 (('club', 'ketchup'), ('clubs', 0.5758280754089355)),
 (('horn', 'knight'), ('horns', 0.5737985968589783)),
 (('club', 'green'), ('clubs', 0.5700738430023193)),
 (('knight', 'jet'), ('jets', 0.5692975521087646)),
 (('horn', 'green'), ('horns', 0.5672204494476318)),
 (('club', 'jet', 'ketchup'), ('jets', 0.5672203302383423)),
 (('duck', 'club'), ('clubs', 0.5643361806869507)),
 (('horn', 'jet', 'ketchup'), ('jets', 0.5624082088470459)),
 (('horn', 'slug'), ('horns', 0.5482915043830872)),
 (('duck', 'club', 'jet'), ('plane', 0.547

In [13]:
combinations_blue = powerset(blue)[10:]

clue_options_blue = [(subset_blue, model.most_similar(
    positive=subset_blue,
    restrict_vocab=50000)[1]) for subset_blue in combinations_blue]

list(sorted(clue_options_blue, key=lambda r: r[1][1], reverse=True))

[(('scientist', 'embassy'), ('diplomat', 0.6457305550575256)),
 (('opera', 'embassy'), ('consulate', 0.6175993084907532)),
 (('chest', 'embassy'), ('consulate', 0.6078716516494751)),
 (('boot', 'embassy'), ('consulate', 0.6076034307479858)),
 (('opera', 'scientist'), ('operas', 0.6048358678817749)),
 (('triangle', 'embassy'), ('embassies', 0.592045783996582)),
 (('opera', 'scientist', 'embassy'), ('diplomat', 0.5914803743362427)),
 (('opera', 'watch'), ('watching', 0.5858166217803955)),
 (('parachute', 'embassy'), ('consulate', 0.5838911533355713)),
 (('square', 'embassy'), ('consulate', 0.5809622406959534)),
 (('chest', 'opera', 'embassy'), ('consulate', 0.5702699422836304)),
 (('triangle', 'scientist'), ('researcher', 0.5678838491439819)),
 (('watch', 'embassy'), ('embassies', 0.5639655590057373)),
 (('chest', 'triangle'), ('abdomen', 0.5638320446014404)),
 (('chest', 'scientist', 'embassy'), ('consulate', 0.559476375579834)),
 (('boot', 'opera'), ('Opera', 0.5537703633308411)),
 (('

In [11]:
model.similar_by_word('parachute', topn=10)

[('parachutes', 0.7980984449386597),
 ('parachutist', 0.562181293964386),
 ('ripcord', 0.5591350197792053),
 ('parachutists', 0.5577532052993774),
 ('parachuting', 0.5507856607437134),
 ('chute', 0.5506881475448608),
 ('rip_cord', 0.5416831374168396),
 ('hang_glider', 0.5298277735710144),
 ('balloon', 0.521361768245697),
 ('glider', 0.5206780433654785)]

SyntaxError: invalid syntax (<ipython-input-15-da518ff03000>, line 1)